In [10]:
import torch
import os

print("---- Environment Validation ----")
print("GPU available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU name:", torch.cuda.get_device_name(0))

for var in ["HF_TOKEN", "HF_USER", "SPACE_NAME"]:
    val = os.environ.get(var)
    print(f"{var}:", " SET" if val else "NOT SET")

wandb_key = os.environ.get("WANDB_API_KEY")
print("WANDB_API_KEY:", " SET" if wandb_key else "Not set")

try:
    import wandb, gradio, huggingface_hub
    print("Dependencies: All imported successfully")
except Exception as e:
    print("Dependencies: Missing:", e)


---- Environment Validation ----
GPU available: False
HF_TOKEN:  SET
HF_USER:  SET
SPACE_NAME:  SET
WANDB_API_KEY:  SET
Dependencies: All imported successfully


In [3]:
from getpass import getpass
import os

print("Paste your Hugging Face token when prompted (it will be hidden)")
hf_token = getpass("Hugging Face token: ")
os.environ['HF_TOKEN'] = hf_token

hf_user = input("Enter your Hugging Face username: ").strip()
space_name = input("Enter Space name (e.g., 'tiny-imagenet-classifier'): ").strip()

os.environ['HF_USER'] = hf_user
os.environ['SPACE_NAME'] = space_name

print("Credentials stored in environment")

Paste your Hugging Face token when prompted (it will be hidden)
Credentials stored in environment
Credentials stored in environment


In [4]:
wandb_entity = input("Enter your W&B entity/username: ").strip()
wandb_project = input("Enter W&B project name: ").strip() or "tiny-imagenet-assignment"

os.environ['WANDB_ENTITY'] = "ir2023"
os.environ['WANDB_PROJECT'] = "tiny-imagenet-assignment"

print(f"Will use artifact: {wandb_entity}/{wandb_project}/resnet18-tiny-imagenet:latest")

Will use artifact: /tiny-imagenet-assignment/resnet18-tiny-imagenet:latest


In [11]:
code = """
import os
import time
import torch
import torch.nn as nn
from PIL import Image
import torchvision.transforms as transforms
from torchvision.models import resnet18
import gradio as gr
import json

MODEL_PATH = "outputs/best_model.pth"

# Load class names
def load_class_names():
    if os.path.exists("class_names.json"):
        with open("class_names.json", "r") as f:
            return json.load(f)
    return {f"class_{i}": f"class_{i}" for i in range(200)}

CLASS_NAMES = load_class_names()

# Download model from W&B if not present
if not os.path.exists(MODEL_PATH):
    try:
        import wandb
        wandb_api_key = os.environ.get("WANDB_API_KEY")
        if wandb_api_key:
            print("Downloading model from W&B...")
            wandb.login(key=wandb_api_key)
            api = wandb.Api()
            artifact_path = os.environ.get("WANDB_ARTIFACT", 
            "ir2023/tiny-imagenet-assignment/resnet18-tiny-imagenet:latest")
            print(f"Downloading: {artifact_path}")
            artifact = api.artifact(artifact_path)
            artifact.download(root="outputs")
            print("Model downloaded from W&B")
        else:
            print("WANDB_API_KEY not set in Space Secrets")
    except Exception as e:
        print(f"Error downloading model: {e}")

# Load model
print("Loading model...")
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

model = resnet18(pretrained=False)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 200)

try:
    checkpoint = torch.load(MODEL_PATH, map_location=device)
    if 'model_state_dict' in checkpoint:
        model.load_state_dict(checkpoint['model_state_dict'])
    else:
        model.load_state_dict(checkpoint)
    print("Model loaded successfully")
except Exception as e:
    print(f"Error loading model: {e}")
    raise

model.to(device)
model.eval()

# Image preprocessing
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                       std=[0.229, 0.224, 0.225])
])

def predict_image(img):
    if img is None:
        return {"error": "No image provided"}
    
    start_time = time.time()
    
    try:
        img_tensor = transform(img).unsqueeze(0).to(device)
        
        with torch.no_grad():
            outputs = model(img_tensor)
            probs = torch.nn.functional.softmax(outputs, dim=1)
            top5_probs, top5_idx = torch.topk(probs, 5)
        
        results = {}
        for prob, idx in zip(top5_probs[0], top5_idx[0]):
            class_idx = int(idx.item())
            class_name = list(CLASS_NAMES.values())[class_idx]
            class_id = list(CLASS_NAMES.keys())[class_idx]
            results[f"{class_name} ({class_id})"] = float(prob.item())
        
        latency = (time.time() - start_time) * 1000.0
        print(f"Prediction time: {latency:.2f}ms")
        
        return results
        
    except Exception as e:
        return {"error": f"Prediction failed: {str(e)}"}

# Create Gradio interface
title = "Tiny ImageNet Classifier"
description = '''
Upload an image to classify it into one of 200 Tiny ImageNet classes.

**Model Details:**
- Architecture: ResNet-18
- Dataset: Tiny ImageNet (200 classes)
- Input Size: 224x224 RGB
- Framework: PyTorch

Model is automatically downloaded from W&B at startup.
'''

iface = gr.Interface(
    fn=predict_image,
    inputs=gr.Image(type="pil", label="Upload Image"),
    outputs=gr.Label(num_top_classes=5, label="Top 5 Predictions"),
    title=title,
    description=description,
    theme="default",
    allow_flagging="never"
)

if __name__ == "__main__":
    iface.launch(server_name="0.0.0.0", server_port=7860)
"""
with open("app.py", "w") as f:
    f.write(code)
print("app.py created successfully")


app.py created successfully


In [12]:
with open("requirements.txt", "w") as f:
    f.write("""torch==2.0.1
    torchvision==0.15.2
    gradio==4.0.0
    Pillow==10.0.0
    wandb>=0.16.0
    huggingface_hub
    git-lfs
    """)

print("Created requirements.txt")


Created requirements.txt


In [13]:
import os
import shutil
from pathlib import Path
from huggingface_hub import HfApi

# Clean up and prepare directory
hf_space_dir = Path('hf_space')
if hf_space_dir.exists():
    shutil.rmtree(hf_space_dir)
hf_space_dir.mkdir()

# Copy required files
shutil.copy('app.py', hf_space_dir / 'app.py')
shutil.copy('requirements.txt', hf_space_dir / 'requirements.txt')

# Copy class_names.json if it exists
if Path('class_names.json').exists():
    shutil.copy('class_names.json', hf_space_dir / 'class_names.json')

# Create and push to Hugging Face Space
token = os.environ.get("HF_TOKEN")
user = os.environ.get("HF_USER")
space = os.environ.get("SPACE_NAME")

if not token or not user or not space:
    raise ValueError("HF_TOKEN, HF_USER or SPACE_NAME not set!")

api = HfApi(token=token)
repo_id = f"{user}/{space}"

print(f"Creating Space: {repo_id}")

try:
    repo_url = api.create_repo(
        repo_id=repo_id,
        repo_type="space",
        space_sdk="gradio",
        exist_ok=True
    )
    print(f"Space created: {repo_url}")
    
    api.upload_folder(
        folder_path=str(hf_space_dir),
        repo_id=repo_id,
        repo_type="space",
        commit_message="Initial commit: Tiny ImageNet Gradio app"
    )
    
    print("Files uploaded successfully!")
    print(f"\nSpace URL: {repo_url}")
    
except Exception as e:
    print(f"Error: {e}")
    raise

Creating Space: rahulmirapala/tiny-imagenet-classifier
Space created: https://huggingface.co/spaces/rahulmirapala/tiny-imagenet-classifier
Space created: https://huggingface.co/spaces/rahulmirapala/tiny-imagenet-classifier
Files uploaded successfully!

Space URL: https://huggingface.co/spaces/rahulmirapala/tiny-imagenet-classifier
Files uploaded successfully!

Space URL: https://huggingface.co/spaces/rahulmirapala/tiny-imagenet-classifier
